In [1]:
import os

In [2]:
%pwd

'c:\\NLP PROJECTS\\Text-Summarization-NLP-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\NLP PROJECTS\\Text-Summarization-NLP-Project'

In [11]:
import os
from dataclasses import dataclass
from pathlib import Path
from textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from textSummarizer.utils.common import read_yaml, create_directories
from textSummarizer.logging import logger

# ✅ Fix 1: Define DataValidationConfig Correctly
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: Path  # ✅ Changed str to Path
    ALL_REQUIRED_FILES: list

# ✅ Fix 2: Properly Initialize ConfigurationManager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=Path(CONFIG_FILE_PATH),  # ✅ Convert to Path
        params_filepath=Path(PARAMS_FILE_PATH)   # ✅ Convert to Path
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        # ✅ Fix 3: Convert Paths Properly
        root_dir = Path(config.root_dir)
        status_file = Path(config.STATUS_FILE)

        create_directories([root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=root_dir,
            STATUS_FILE=status_file,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES,
        )

        return data_validation_config

# ✅ Fix 4: Corrected Class Name (Spelling Mistake)
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        try:
            validation_status = True
            dataset_path = Path("artifacts/data_ingestion/samsum_dataset")

            if not dataset_path.exists():
                logger.error(f"Dataset path {dataset_path} does not exist.")
                return False

            all_files = os.listdir(dataset_path)

            for file in self.config.ALL_REQUIRED_FILES:
                if file not in all_files:
                    validation_status = False
                    logger.error(f"Missing file: {file}")
            
            # ✅ Fix 5: Use Path for Status File
            with self.config.STATUS_FILE.open('w') as f:
                f.write(f"Validation status: {validation_status}")

            logger.info(f"Validation status: {validation_status}")
            return validation_status

        except Exception as e:
            raise RuntimeError(f"File validation error: {e}")

# ✅ Running the Code Safely
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise RuntimeError(f"Data validation failed: {e}")


[2025-03-22 17:40:24,548: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-22 17:40:24,556: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-22 17:40:24,559: INFO: common: created directory at: artifacts]
[2025-03-22 17:40:24,561: INFO: common: created directory at: artifacts\data_validation]
[2025-03-22 17:40:24,564: INFO: 1378558350: Validation status: True]
